This notebook describes a backup plan for downloading data manually, should the `yfinance` package stop working (as it has, in the past).

You would need to go to Yahoo! Finance website and download each ETF and currency file manually and put them in appropriate folder.

The historical data should cover as long a time period as possible and come with a daily frequency.

In [1]:
import os
import pandas as pd
from datetime import date

In [2]:
START_DATE = date(2001, 1, 1)
END_DATE = date(2023, 12, 31)

In [3]:
def read_yahoo_csv(filepath):
    df = pd.read_csv(filepath)[["Date", "Adj Close"]]
    df["Date"] = pd.to_datetime(df["Date"])

    column_name = ".".join(os.path.basename(filepath).split(".")[:-1])

    df.rename(columns={"Adj Close": column_name}, inplace=True)
    return df


def read_yahoo_folder(folderpath, start, end):
    df = pd.DataFrame({"Date": pd.date_range(start, end, freq="D")})

    with os.scandir(folderpath) as entries:
        for entry in entries:
            df = df.merge(read_yahoo_csv(entry), on="Date", how="left")

    df.set_index("Date", inplace=True)
    return df

In [4]:
etf_df = read_yahoo_folder("data/etfs/", START_DATE, END_DATE)
exchange_rates_df = read_yahoo_folder("data/exchange-rates/", START_DATE, END_DATE)